In [ ]:
import bpy
from IPython.display import display
from anywidget_3dviewer import Viewer
from pathlib import Path

def make_gltf_model():

    bpy.ops.preferences.addon_enable(module='io_scene_gltf2')

    bpy.ops.object.select_all(action="DESELECT")
    bpy.ops.object.select_by_type(type="MESH")
    bpy.ops.object.delete()
    bpy.ops.mesh.primitive_cube_add(size=2, align="WORLD", location=(0, 1, 1))

    cube = bpy.context.active_object

    material = bpy.data.materials.new(name="BlueMaterial")
    cube.data.materials.append(material)
    material.use_nodes = True
    nodes = material.node_tree.nodes

    for node in nodes:
        nodes.remove(node)

    bsdf = nodes.new(type="ShaderNodeBsdfPrincipled")
    bsdf.location = (400, 0)
    bsdf.inputs["Base Color"].default_value = (1, 0.5, 0, 1)
    output = nodes.new(type="ShaderNodeOutputMaterial")
    output.location = (600, 0)

    material.node_tree.links.new(bsdf.outputs["BSDF"], output.inputs["Surface"])


    gltf_path = "./model_cube.gltf"
    bpy.ops.object.select_all(action='DESELECT')
    cube.select_set(True)
    bpy.ops.export_scene.gltf(filepath=gltf_path, export_format='GLTF_EMBEDDED', use_selection=True)


    gltf_path = Path() / "model_cube.gltf"
    data = gltf_path.read_bytes()
    gltf_path.unlink()

    return data

data = make_gltf_model()

display(Viewer(gltf_data=data))

In [ ]:
bsdf.inputs["Base Color"].default_value = (1, 0.5, 0, 1)

In [ ]:
bsdf.inputs["Base Color"].default_value = (1, 0.5, 0, 1)